#  迭代

In [2]:
from __future__ import division
import numpy as np
import snap
import matplotlib.pyplot as plt
import datetime    


Bad key "text.kerning_factor" on line 4 in
E:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
def sim_truth(g,c,T):
    '''该函数通过迭代计算g的simrank，其中c是衰减因子，T为轮数。返回simrank矩阵，使用MSE记录矩阵变化'''
    N=g.GetNodes()
    # M=g.GetEdges()
    s=np.eye(N,N)
    for t in range(T):
        temp=np.eye(N,N)
        for i in range(N):
            Ni=g.GetNI(i)
            DegOfI=Ni.GetInDeg()  #是一个整数
            for j in range(i+1,N):
                Nj=g.GetNI(j)
                DegOfJ=Nj.GetInDeg()
                InEdgeOfJ=Nj.GetInEdges()
                InEdgeOfI = Ni.GetInEdges()  # 构思产生
                # 对于邻居i和j的每个节点，将其simrank值相加
                # InEdgeOfI and InEdgeOfJ are int generator
                val=0.
                if DegOfI and DegOfJ:
                    for u in InEdgeOfI:
                        for v in InEdgeOfJ:
                            val+=s[u][v]
                    val*=(c/DegOfI/DegOfJ)
                temp[i][j]=val
                temp[j][i]=val
        s=temp
    return s  

In [20]:
def main():
    start=datetime.datetime.now()
    g=snap.LoadEdgeList(snap.PNGraph,"G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/email-Eu-core.txt")
    c=0.6
    T=10
    s=sim_truth(g,c,T)
    np.save('G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/simrank.ipynb',s)
    end=datetime.datetime.now()
    print('total time: '+str((end-start).seconds)+' seconds') 

In [21]:
if __name__=="__main__":
    main()

AttributeError: module 'snap' has no attribute 'LoadEdgeList'

# 运行

In [14]:
from __future__ import division
import numpy as np
import snap
import datetime 

In [15]:
def subroute(g,begin,l,visited):
    factor=1.
    walker=begin
    for j in range(l):
        Ni = g.GetNI(walker)
        OutNeibors = list(Ni.GetOutEdges())
        OutDeg = Ni.GetOutDeg()
        if OutDeg==0:
            break
        walker=np.random.choice(OutNeibors)
        if walker not in visited:
            InDeg=g.GetNI(walker).GetInDeg()
            factor*=OutDeg/InDeg
        else:
            break
    else:
        return factor,walker
    return 0,walker   

In [16]:
def ssRWalk(u,c,L,R,g):
    N=g.GetNodes()
    ss=np.zeros(N)
    ss[u]=R
    for r in range(R):
        walker=u
        visited=set()
        visited.add(walker)
        for j in range(1,L+1):
            Ni=g.GetNI(walker)
            InNeibors=list(Ni.GetInEdges())
            InDeg=Ni.GetInDeg()
            if InDeg==0:
                break
            walker=np.random.choice(InNeibors)
            if walker in visited:
                break
            visited.add(walker)
            factor,end=subroute(g,walker,j,visited)
            ss[end]+=factor*(c**j)
    ss/=R
    return ss   

In [17]:
def my_walk(g,c,L,R):
    N=g.GetNodes()
    s=np.zeros((N,N))
    for i in range(N):
        s[i]=ssRWalk(i,c,L,R,g)
    return s 

In [25]:
def main():
    start=datetime.datetime.now()
    g = snap.LoadEdgeList(snap.PNGraph, 'G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/email-Eu-core.txt')
    N=g.GetNodes()
    c=0.6
    L=6
    R=1000
    s=my_walk(g,c,L,R)
    np.save('G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/simByMW.ipynb',s)
    end=datetime.datetime.now()
    print("the process time is ",(end-start).seconds,"seconds")  

In [26]:
if __name__=="__main__":
    main()

AttributeError: module 'snap' has no attribute 'LoadEdgeList'

# 原始运行

In [22]:
from __future__ import division
import numpy as np
import snap
import datetime 

In [23]:
def path_sample(g,source,L):
    path=list()
    path.append(source)
    walker=source
    for i in range(L):
        Ni=g.GetNI(walker)
        InDegree=Ni.GetInDeg()
        InNeibors=list(Ni.GetInEdges())
        if InDegree==0:
            break
        walker=np.random.choice(InNeibors)
        path.append(walker)
    return path 

In [24]:
def naive_walk(g,c,L,R):
    N=g.GetNodes()
    s=np.zeros((N,N))
    for r in range(R):
        print("round ",r)
        all_path=list()  ## 从图g的每个节点中抽取一条长度为L的路径
        for n in range(N):
            all_path.append(path_sample(g,n,L))
        for i in range(N):
            for j in range(i,N):
                pathi=all_path[i]
                pathj=all_path[j]
                sd=0.
                for k in range(min(len(pathi),len(pathj))):
                    if pathi[k]==pathj[k]:
                        sd=c**k
                        break
                s[i,j]+=sd
                s[j,i]+=sd
    s/=R
    return s  

In [29]:
def main():
    start = datetime.datetime.now()
    g = snap.LoadEdgeList(snap.PNGraph, 'G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/email-Eu-core.txt')
    N = g.GetNodes()
    c = 0.6
    L = 6
    R = 1000
    s = naive_walk(g,c,L,R)
    np.save('G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/simByNaive.ipynb',s)
    end=datetime.datetime.now()
    print("the process time is",(end-start).seconds,"seconds") 

In [30]:
if __name__ == '__main__':
    main() 

AttributeError: module 'snap' has no attribute 'LoadEdgeList'

# 实验

In [34]:
from __future__ import division
import my_walk as mw
import naive_walk as nw
import numpy as np
import datetime
import snap 

ModuleNotFoundError: No module named 'my_walk'

In [ ]:
def var_L(g,c,L_list,R):
    N=g.GetNodes()
    d=len(L_list)
    naive_ret=np.zeros((d,N,N))
    my_ret=np.zeros((d,N,N))
    for i,L in enumerate(L_list):
        sn=nw.naive_walk(g,c,L,R)
        sm=mw.my_walk(g,c,L,R)
        naive_ret[i,:,:]=sn
        my_ret[i,:,:]=sm
    return naive_ret,my_ret

In [ ]:
def var_R(g,c,L,R_list):
    N=g.GetNodes()
    d=len(R_list)
    naive_ret=np.zeros((d,N,N))
    my_ret=np.zeros((d,N,N))
    for i,R in enumerate(R_list):
        sn=nw.naive_walk(g,c,L,R)
        sm=mw.my_walk(g,c,L,R)
        naive_ret[i,:,:]=sn
        my_ret[i,:,:]=sm
    return naive_ret,my_ret

In [ ]:
def main():
    start = datetime.datetime.now()
    g = snap.LoadEdgeList(snap.PNGraph, 'G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/email-Eu-core.txt')
    c = 0.6
    L_list=range(1,7)
    R_list=range(200,1200,200)
    print 'computing with L variating...'
    a,b=var_L(g,c,L_list,1000)
    print 'computing with R variating...'
    c,d=var_R(g,c,6,R_list)

    ## save the results to the disk. 'n' represents naive_walk result matrix and
    # 'm' represents my_walk result matrix
    np.savez('G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/var_L.npz',n=a,m=b)
    np.savez('G:/CDO/海上风电场/蒙特卡洛模拟/simrank-master/var_R.npz',n=c,m=d)

    end=datetime.datetime.now()
    print 'total time: '+str((end-start).seconds)+' seconds'


In [ ]:
if __name__ == '__main__':
    main()

In [32]:
import numpy as np
import matplotlib.pyplot as plt

In [33]:
def plot_ME(truth,naive,my):
    '''真值为simrank真值矩阵，通过迭代求解得到，naive和my都是一个P*N*N矩阵，其中P是标点数'''
    k,N,_=naive.shape